## 1. Imports et Configuration

In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import requests

# Import du module d'analyse
from analyses.statistical_analysis import StatisticalAnalysis
from analyses.data_cleaning import DataCleaning

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

API_BASE_URL = "http://localhost:8000"

## 2. Chargement des Données

In [ ]:
# Fonction helper pour charger les données
def load_data():
    """Charge et nettoie les données"""
    cleaner = DataCleaning()
    
    # Charger depuis l'API ou fichiers CSV
    try:
        response = requests.get(f"{API_BASE_URL}/accidents?limit=10000")
        df = pd.DataFrame(response.json())
        print(f"Données chargées via API: {len(df)} accidents")
    except:
        # Fallback: charger depuis CSV
        df = pd.read_csv('../data/clean/accidents_clean.csv')
        print(f"Données chargées depuis CSV: {len(df)} accidents")
    
    return df

df = load_data()
df.head()

## 3. Initialisation du Module Statistique

In [ ]:
# Créer l'instance d'analyse statistique
stat_analyzer = StatisticalAnalysis(df)
print("Module statistique initialisé")
print(f"Dataset: {len(df)} observations, {len(df.columns)} variables")

## 4. Test Chi2 - Associations entre Variables Catégorielles

In [ ]:
# Test Chi2: Gravité vs Conditions météo
if 'grav' in df.columns and 'atm' in df.columns:
    chi2_result = stat_analyzer.chi2_test('grav', 'atm')
    
    print("=" * 60)
    print("TEST CHI2: Gravité vs Conditions Atmosphériques")
    print("=" * 60)
    print(f"Chi2 statistic: {chi2_result['statistic']:.4f}")
    print(f"P-value: {chi2_result['p_value']:.6f}")
    print(f"Degrés de liberté: {chi2_result['dof']}")
    
    if chi2_result['p_value'] < 0.05:
        print("\n[SIGNIFICATIF] Association SIGNIFICATIVE (p < 0.05)")
    else:
        print("\n[NON SIGNIFICATIF] Pas d'association significative (p >= 0.05)")
    
    # Visualisation du tableau de contingence
    contingency = pd.crosstab(df['grav'], df['atm'])
    plt.figure(figsize=(10, 6))
    sns.heatmap(contingency, annot=True, fmt='d', cmap='YlOrRd')
    plt.title('Tableau de Contingence: Gravité vs Conditions Météo')
    plt.tight_layout()
    plt.show()

## 5. ANOVA - Comparaison de Groupes

In [ ]:
# ANOVA: Nombre de victimes par type de collision
if 'col' in df.columns and 'nb_victimes' in df.columns:
    anova_result = stat_analyzer.anova_test('nb_victimes', 'col')
    
    print("=" * 60)
    print("ANOVA: Nombre de Victimes par Type de Collision")
    print("=" * 60)
    print(f"F-statistic: {anova_result['f_statistic']:.4f}")
    print(f"P-value: {anova_result['p_value']:.6f}")
    
    if anova_result['p_value'] < 0.05:
        print("\n[SIGNIFICATIF] Différences SIGNIFICATIVES entre les groupes")
    else:
        print("\n[NON SIGNIFICATIF] Pas de différence significative")
    
    # Visualisation boxplot
    plt.figure(figsize=(12, 6))
    df.boxplot(column='nb_victimes', by='col', figsize=(12, 6))
    plt.title('Distribution des Victimes par Type de Collision')
    plt.suptitle('')
    plt.xlabel('Type de Collision')
    plt.ylabel('Nombre de Victimes')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 6. Corrélations de Pearson

In [ ]:
# Matrice de corrélation complète
corr_matrix = stat_analyzer.correlation_analysis()

print("=" * 60)
print("CORRÉLATIONS SIGNIFICATIVES (|r| > 0.3)")
print("=" * 60)

# Afficher les corrélations fortes
strong_corr = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        corr_val = corr_matrix.iloc[i, j]
        if abs(corr_val) > 0.3:
            var1 = corr_matrix.columns[i]
            var2 = corr_matrix.columns[j]
            strong_corr.append((var1, var2, corr_val))

if strong_corr:
    for var1, var2, corr in sorted(strong_corr, key=lambda x: abs(x[2]), reverse=True):
        print(f"{var1} <-> {var2}: r = {corr:.3f}")
else:
    print("Aucune corrélation forte détectée")

# Heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matrice de Corrélation (Pearson)')
plt.tight_layout()
plt.show()

## 7. Statistiques Descriptives Avancées

In [ ]:
# Statistiques détaillées
desc_stats = stat_analyzer.descriptive_statistics()

print("=" * 60)
print("STATISTIQUES DESCRIPTIVES")
print("=" * 60)
print(desc_stats)

## 8. Intervalles de Confiance

In [ ]:
# Intervalles de confiance pour une variable numérique
if 'nb_victimes' in df.columns:
    ci = stat_analyzer.confidence_interval('nb_victimes', confidence=0.95)
    
    print("=" * 60)
    print("INTERVALLE DE CONFIANCE (95%)")
    print("=" * 60)
    print(f"Moyenne: {ci['mean']:.4f}")
    print(f"Écart-type: {ci['std']:.4f}")
    print(f"IC 95%: [{ci['lower']:.4f}, {ci['upper']:.4f}]")
    print(f"Taille échantillon: {ci['n']}")

## 9. Analyse Temporelle

In [ ]:
# Évolution temporelle des accidents
if 'an' in df.columns:
    temporal = df.groupby('an').agg({
        'Num_Acc': 'count',
        'grav': lambda x: (x >= 3).sum() if 'grav' in df.columns else 0
    }).rename(columns={'Num_Acc': 'Total', 'grav': 'Graves'})
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Total accidents
    temporal['Total'].plot(ax=axes[0], marker='o', linewidth=2, color='steelblue')
    axes[0].set_title('Évolution du Nombre Total d\'Accidents', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Année')
    axes[0].set_ylabel('Nombre d\'accidents')
    axes[0].grid(True, alpha=0.3)
    
    # Accidents graves
    temporal['Graves'].plot(ax=axes[1], marker='s', linewidth=2, color='darkred')
    axes[1].set_title('Évolution des Accidents Graves', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Année')
    axes[1].set_ylabel('Nombre d\'accidents graves')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Test de tendance
    from scipy.stats import linregress
    years = temporal.index.values
    values = temporal['Total'].values
    slope, intercept, r_value, p_value, std_err = linregress(years, values)
    
    print("\n=" * 60)
    print("ANALYSE DE TENDANCE")
    print("=" * 60)
    print(f"Pente: {slope:.2f} accidents/an")
    print(f"R²: {r_value**2:.4f}")
    print(f"P-value: {p_value:.6f}")
    
    if p_value < 0.05:
        trend = "décroissance" if slope < 0 else "croissance"
        print(f"\n[SIGNIFICATIF] Tendance significative de {trend}")
    else:
        print("\n[NON SIGNIFICATIF] Pas de tendance significative")

## 10. Rapport Statistique Final

In [ ]:
print("=" * 60)
print("RÉSUMÉ DES ANALYSES STATISTIQUES")
print("=" * 60)
print(f"Dataset: {len(df)} observations")
print(f"Variables numériques: {len(df.select_dtypes(include=[np.number]).columns)}")
print(f"Variables catégorielles: {len(df.select_dtypes(include=['object', 'category']).columns)}")
print("\nAnalyses statistiques terminées!")
print("\nProchaines étapes:")
print("  → Notebook 03: Modélisation ML")
print("  → Notebook 04: Visualisations avancées")